In [10]:
from pyspark.sql import SparkSession

# Azure storage access info
blob_account_name = 'pocdashboard01' # replace with your blob name
blob_container_name = 'dashboardpoc' # replace with your container name
blob_relative_path = 'dashboarddata' # replace with your relative folder path
linked_service_name = 'AzureBlobStorage1' # replace with your linked service name

blob_sas_token = mssparkutils.credentials.getConnectionStringOrCreds(linked_service_name)

# Allow SPARK to access from Blob remotely

wasb_path = 'wasbs://%s@%s.blob.core.windows.net/%s' % (blob_container_name, blob_account_name, blob_relative_path)

spark.conf.set('fs.azure.sas.%s.%s.blob.core.windows.net' % (blob_container_name, blob_account_name), blob_sas_token)
print('Remote blob path: ' + wasb_path)

StatementMeta(pocdashboard, 118, 3, Finished, Available)

Remote blob path: wasbs://dashboardpoc@pocdashboard01.blob.core.windows.net/dashboarddata


In [12]:
dataframe1 = spark.read.option('header',True).option("escape","\"").option("multiline", True).csv(wasb_path + '/New Project Master - Segregation.csv')
dataframe1.show()
dataframe1.createOrReplaceTempView("dataframe1")

StatementMeta(pocdashboard, 118, 5, Finished, Available)

+--------------------+----------------+-----------------+----------+----------------------+----------+----------+-----------------+-------------+----------------------+----------+----------+------------+-------------+--------------+--------------------+--------------+--------+--------------------+--------+--------------+----------------+--------+--------------------+--------------+------------------+------------------+------------+-------------+----+--------------+-----------+---------------+--------+------+--------+----------------+-----------+------------+----------+--------------------+--------------------+----------+---------------+-------------+--------+-------+------------+--------------+---------------+-----------+-------+
|        Project Name|          Client| Category Details|MPD Status|Project Classification|Start Date|  End Date|Potential Revenue|Total Revenue|Total Rev + 40% Margin|       MRR|No. of FTE|Working Days|     Industry|        Region|    Technical Skills|Project

In [13]:
from pyspark.sql.functions import to_date,col
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")
Query = spark.sql("""SELECT dataframe1.`Project Name` AS Project_Name,
                    NVL(dataframe1.`Client`,'N/A') AS Client,
                    NVL(dataframe1.`Category Details`,'N/A') AS Category_Details,
                    NVL(dataframe1.`MPD Status`,'N/A') AS MPD_Status,
                    NVL(dataframe1.`Project Classification`,'N/A') AS Project_Classification,
                    NVL(dataframe1.`Start Date`,NULL) AS Start_Date,
                    NVL(dataframe1.`End Date`,NULL) AS End_Date,
                    if(dataframe1.`Potential Revenue` IS not null,if(SUBSTRING(dataframe1.`Potential Revenue`,1,1) ='(', CAST(regexp_replace(RTRIM(')',LTRIM("($",dataframe1.`Potential Revenue`)), ',','') AS DECIMAL(16,2)) * -1  ,CAST(regexp_replace(LTRIM('$',dataframe1.`Potential Revenue`), ',','')AS DECIMAL(16,2))),0) AS Potential_Revenue,                   
                    if(dataframe1.`Total Revenue` IS not null,if(SUBSTRING(dataframe1.`Total Revenue`,1,1) ='(', CAST(regexp_replace(RTRIM(')',LTRIM("($",dataframe1.`Total Revenue`)), ',','') AS DECIMAL(16,2)) * -1  ,CAST(regexp_replace(LTRIM('$',dataframe1.`Total Revenue`), ',','')AS DECIMAL(16,2))),0) AS Total_Revenue,
                    ROUND(if(dataframe1.`Total Revenue` IS not null,if(SUBSTRING(dataframe1.`Total Revenue`,1,1) ='(',ROUND((CAST(regexp_replace(RTRIM(')',LTRIM("($",dataframe1.`Total Revenue`)), ',','') AS DECIMAL(16,2)) +  CAST(regexp_replace(RTRIM(')',LTRIM("($",dataframe1.`Total Revenue`)), ',','') AS DECIMAL(16,2)) * .4),2) * -1,CAST(regexp_replace(LTRIM('$',dataframe1.`Total Revenue`),',','') AS DECIMAL(16,2)) + CAST(regexp_replace(LTRIM('$',dataframe1.`Total Revenue`),',','') AS DECIMAL(16,2)) * .4),0),2) AS Total_Rev_40_Margin,
                    ROUND(if(dataframe1.`Month Count` != 0,if(SUBSTRING(dataframe1.`Total Revenue`,1,1) ='(',CAST(regexp_replace(RTRIM(')',LTRIM("($",dataframe1.`Total Revenue`)), ',','') AS DECIMAL(16,2)) / CAST(dataframe1.`Month Count` AS DECIMAL(16,2)) * -1,ROUND(CAST(regexp_replace(LTRIM('$',dataframe1.`Total Revenue`), ',','')AS DECIMAL(16,2)) / CAST(dataframe1.`Month Count` AS DECIMAL(16,2)),2)),0),2) AS MRR,
                    NVL(CAST(dataframe1.`No. of FTE` AS DECIMAL(16,2)),CAST('0' AS DECIMAL(16,0))) AS No_of_FTE,
                    NVL(CAST(dataframe1.`Working Days` AS DECIMAL(16,0)),CAST('0' AS DECIMAL(16,0))) AS Working_Days,
                    NVL(dataframe1.`Industry`,'N/A') AS Industry,
                    NVL(dataframe1.`Region`,'N/A') AS Region,
                    NVL(dataframe1.`Technical Skills`,'N/A') AS Technical_Skills,
                    NVL(dataframe1.`Project Status`,'N/A') AS Project_Status,
                    NVL(dataframe1.`Comments`,'N/A') AS Comments,
                    NVL(dataframe1.`Market Unit`,'N/A') AS Market_Unit,
                    NVL(dataframe1.`CRM Link`,'N/A') AS CRM_Link,
                    NVL(dataframe1.`Project Stages`,'N/A') AS Project_Stages,
                    NVL(dataframe1.`Project Location`,'N/A') AS Project_Location,
                    NVL(dataframe1.`Channel`,'N/A') AS Channel,
                    NVL(dataframe1.`DCSO Approved (ToRs)`,'N/A') AS DCSO_Approved_ToRs,
                    NVL(dataframe1.`Project POC`,'N/A') AS Project_POC,
                    NVL(dataframe1.`Owner`,'N/A') AS Owner,
                    NVL(dataframe1.`Primary Contact`,'N/A') AS Primary_Contact,
                    NVL(dataframe1.`Rate Type`,'N/A') AS Rate_Type,
                    NVL(dataframe1.`Project Owner`,'N/A') AS Project_Owner,
                    NVL(dataframe1.`WBS`,'N/A') AS WBS,
                    NVL(dataframe1.`Opportunity ID`,'N/A') AS Opportunity_ID,
                    CAST(NVL(CAST(MONTHS_BETWEEN(to_date(dataframe1.`Start Date`,"MM/dd/yyyy"),to_date(dataframe1.`End Date`,"MM/dd/yyyy")) AS INT) * -1,0) AS DECIMAL(16,0)) AS Month_Count,
                    NVL(dataframe1.`Term of Project`,'N/A') AS Term_of_Project,
                    NVL(dataframe1.`FY Start`,'N/A') AS FY_Start,
                    CAST(if(date_format(to_date(dataframe1.`End Date`,"MM/dd/yyyy"),"M") < 9,date_format(to_date(dataframe1.`End Date`,"MM/dd/yyyy"),"yyyy") ,date_format(to_date(dataframe1.`End Date`,"MM/dd/yyyy"),"yyyy") + 1) AS INT) AS FY_End,
                    NVL(dataframe1.`CMS Type`,'N/A') AS CMS_Type,
                    NVL(dataframe1.`New or Extension`,'N/A') AS New_or_Extension,
                    NVL(dataframe1.`C365 Status`,'N/A') AS C365_Status,
                    NVL(dataframe1.`Region (CMS)`,'N/A') AS Region_CMS,
                    if(dataframe1.`MRR old` IS not null,if(SUBSTRING(dataframe1.`MRR old`,1,1) ='(', CAST(regexp_replace(RTRIM(')',LTRIM("($",dataframe1.`MRR old`)), ',','') AS DECIMAL(16,2)) * -1  ,CAST(regexp_replace(LTRIM('$',dataframe1.`MRR old`), ',','')AS DECIMAL(16,2))),0) AS MRR_old,
                    NVL(dataframe1.`Non-Delivery Project`,'N/A') AS Non_Delivery_Project,
                    NVL(dataframe1.`Link`,'N/A') AS Link,
                    NVL(CAST(dataframe1.`MonthCount` AS DECIMAL(16,0)),CAST('0' AS DECIMAL(16,0))) AS MonthCount,
                    NVL(dataframe1.`Orig Start Date`,'N/A') AS Orig_Start_Date,
                    NVL(dataframe1.`Orig End Date`,'N/A') AS Orig_End_Date,
                    NVL(CAST(dataframe1.`Orig FTE` AS DECIMAL(16,2)),CAST('0' AS DECIMAL(16,0))) AS Orig_FTE,
                    NVL(dataframe1.`MPD MPE`,'N/A') AS MPD_MPE
                    from dataframe1""")
Query.createOrReplaceTempView("Query")
display(Query)

StatementMeta(pocdashboard, 118, 6, Finished, Available)

SynapseWidget(Synapse.DataFrame, e6a34d91-aafb-4f2e-8552-4994196cfe11)

In [14]:
from pyspark.sql.functions import to_date,col,date_format,regexp_replace, udf
from pyspark.sql.types import StringType
def termofproject(month):
    if month < 3:
        return "Short Term"
    elif month < 6:
        return "Medium Term"
    else:
        return "Long Term"

termofproject_udf = udf(termofproject,StringType())

spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")
Query1 = Query.withColumn("Term_of_Project", termofproject_udf(col("Month_Count"))) \
            .withColumn("End_Date", to_date(col("End_Date"),"MM/dd/yyyy")) \
            .withColumn("Start_Date", to_date(col("Start_Date"),"MM/dd/yyyy")) \
            .withColumn("Orig_Start_Date",to_date(col("Orig_Start_Date"),"MM/dd/yyyy")) \
            .withColumn("Orig_End_Date",to_date(col("Orig_End_Date"),"MM/dd/yyyy")) \
            .withColumn("DCSO_Approved_ToRs",to_date(col("DCSO_Approved_ToRs"),"MM/dd/yyyy")) \
            .withColumn("FY_Start",date_format(regexp_replace(col("FY_Start"),',',''),"yyyy")) \
            .withColumn("FY_End",date_format(regexp_replace(col("FY_End"),',',''),"yyyy")) 
Query1.createOrReplaceTempView("Query1")
display(Query1)

StatementMeta(pocdashboard, 118, 7, Finished, Available)

SynapseWidget(Synapse.DataFrame, ac33dc9b-86a4-459f-bea7-91dccbc062b6)

In [15]:
pdf = Query1.toPandas()
print(pdf)

StatementMeta(pocdashboard, 118, 8, Finished, Available)

                    Project_Name                Client  \
0         5G and IoT Exploration              Internal   
1         5G and IoT Exploration              Internal   
2                 Accenture Labs              Internal   
3    Accenture MD&I/Digital Grid              Internal   
4                          ADAPT                 ADAPT   
..                           ...                   ...   
511                Sight Machine              Internal   
512         JM Family Enterprise  JM Family Enterprise   
513                   Ross Store            Ross Store   
514                    Euroclear             Euroclear   
515                   Coles FY24                 Coles   

              Category_Details MPD_Status Project_Classification  Start_Date  \
0            Completed Revenue  Completed      Intelligent Store  2022-06-13   
1            Completed Revenue  Completed      Intelligent Store  2022-09-01   
2            Completed Revenue  Completed       Cloud Formation

In [16]:
import pyodbc

server = 'mpddashboard.database.windows.net'
database = 'POC_Dashboard'
username = 'mpddashboardpoc'
password = 'internal01!'

cnxn = pyodbc.connect('DRIVER={ODBC Driver 18 for SQL Server};SERVER=' + server + ';DATABASE='+database+';UID=' + username + ';PWD=' + password)
cur = cnxn.cursor()
# cnxn.execute('EXEC spInsertEmp')
# cnxn.commit()

# cnxn.close

StatementMeta(pocdashboard, 118, 9, Finished, Available)

In [17]:
for index, row in pdf.iterrows():
    cur.execute("INSERT INTO dbo.New_Project_Master_Segregation2(Project_Name, Client, Category_Details, MPD_Status, Project_Classification, Start_Date, End_Date, Potential_Revenue, Total_Revenue, Total_Rev_40_Margin, MRR, No_of_FTE, Working_Days, Industry, Region, Technical_Skills, Project_Status, Comments, Market_Unit, CRM_Link, Project_Stages, Project_Location, Channel, DCSO_Approved_ToRs, Project_POC, Owner, Primary_Contact, Rate_Type, Project_Owner, WBS, Opportunity_ID, Month_Count, Term_of_Project, FY_Start, FY_End, CMS_Type, New_or_Extension, C365_Status, Region_CMS, MRR_old, Non_Delivery_Project, Link, MonthCount, Orig_Start_Date, Orig_End_Date, Orig_FTE, MPD_MPE) values(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", row.Project_Name, row.Client, row.Category_Details, row.MPD_Status, row.Project_Classification, row.Start_Date, row.End_Date, row.Potential_Revenue, row.Total_Revenue, row.Total_Rev_40_Margin,row.MRR,row.No_of_FTE,row.Working_Days,row.Industry,row.Region,row.Technical_Skills,row.Project_Status,row.Comments,row.Market_Unit,row.CRM_Link,row.Project_Stages,row.Project_Location,row.Channel,row.DCSO_Approved_ToRs, row.Project_POC,row.Owner, row.Primary_Contact, row.Rate_Type, row.Project_Owner, row.WBS, row.Opportunity_ID, row.Month_Count, row.Term_of_Project, row.FY_Start, row.FY_End, row.CMS_Type, row.New_or_Extension,row.C365_Status,row.Region_CMS,row.MRR_old,row.Non_Delivery_Project,row.Link,row.MonthCount,row.Orig_Start_Date,row.Orig_End_Date,row.Orig_FTE,row.MPD_MPE)
cnxn.commit()

StatementMeta(pocdashboard, 118, 10, Finished, Available)